# Load and downsample

In [35]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.transform import Affine, rowcol

# Paths
tif_path = '/Applications/Emmanuel Zheng/habsim/data/gmted_tif/mn30_grd.tif'
npy_path = '/Applications/Emmanuel Zheng/habsim/data/worldelev.npy'

# Open TIFF and downsample by factor of 2 using bilinear interpolation
with rasterio.open(tif_path) as src:
    data = src.read(
        1,  # first band
        out_shape=(src.height // 2, src.width // 2),
        resampling=Resampling.bilinear
    ).astype(np.float32)
    
    # Update transform to match downsampled array
    transform = src.transform * Affine.scale(src.width / data.shape[1], src.height / data.shape[0])

# Save downsampled array
np.save(npy_path, data, allow_pickle=False)
print(f"Saved downsampled array with shape {data.shape} to {npy_path}")

Saved downsampled array with shape (10440, 21600) to /Applications/Emmanuel Zheng/habsim/data/worldelev.npy


# Test

In [ ]:
# Load for use
data = np.load(npy_path, mmap_mode='r')
rows, cols = data.shape

def getElevation(lat, lon):
    """
    Returns interpolated elevation (meters) for given lat/lon using downsampled array.
    """
    # Convert lat/lon to fractional row/col
    row_f, col_f = rowcol(transform, lon, lat, op=float)

    # Clamp to valid range
    row_f = np.clip(row_f, 0, rows - 1)
    col_f = np.clip(col_f, 0, cols - 1)

    # Integer indices and fractional offsets
    row0 = int(np.floor(row_f))
    col0 = int(np.floor(col_f))
    row1 = min(row0 + 1, rows - 1)
    col1 = min(col0 + 1, cols - 1)
    dr = row_f - row0
    dc = col_f - col0

    # Bilinear interpolation
    v00 = data[row0, col0]
    v10 = data[row0, col1]
    v01 = data[row1, col0]
    v11 = data[row1, col1]
    v_top = v00 * (1 - dc) + v10 * dc
    v_bottom = v01 * (1 - dc) + v11 * dc
    elev = v_top * (1 - dr) + v_bottom * dr

    return float(max(0, elev))

# Example
lat, lon = 37.4393, -121.5769
elevation = getElevation(lat, lon)
print(f"Elevation at ({lat}, {lon}) = {elevation:.2f} m")